In [3]:
from tokenizer import Tokenizer 

tokenizer = Tokenizer.from_files(vocab_path="vocab.json",merges_path="merges.txt",special_tokens=["<|endoftext|>"])

In [5]:
type(tokenizer.vocab)

dict

In [6]:
len(tokenizer.vocab)

1001

In [ ]:
import torch.nn as nn
import torch
class My_Embedding(nn.Module):
    def __init__(self,vocab_size,embedding_dim=10):
        super().__init__()
        
        
        self.weight =nn.Parameter(torch.empty(vocab_size,embedding_dim))
        
        nn.init.trunc_normal_(self.weight,mean=0,std=0.02,a=-2,b=2)
        
    def forward(self,x):
        return self.weight[x]
    

class Model(nn.Module):
    def __init__(self,vocab_size,embedding_dim=10):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        
    def forward(self,x):
        return self.embedding(x)
    
    
embedding = Model(vocab_size=1001,embedding_dim=10)

input = "hello world"

input_ids = tokenizer.encode(input)

input_ids = torch.tensor(input_ids)

input = embedding(input_ids)

input

sim = input[0]@input[1]
sim



tensor(0.3391, grad_fn=<DotBackward0>)